In [2]:
# Import of basic libraries
import tensorflow as tf
from glob import glob
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
from builtins import range, input
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from pathlib import Path
from random import shuffle
from cbir import *
from grad_cam import *
from PIL import Image

In [3]:
IMG_SIZE = 224

def process_data(image_list, IMG_SIZE):
    data_df = []
    for img in image_list:
        img = cv2.imread(img, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data_df.append([np.array(img)])
    shuffle(data_df)
    return data_df

def manual_pre_process(dir, IMG_SIZE):
    data_df = process_data(dir, IMG_SIZE)
    X = np.array([i[0] for i in data_df]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    return X

In [ ]:
#-------------------------------------------------- Simple CBIR ---------------------------------------------------------------#

# Calculate the distances of the five more similar images
# Read image features
fe = FeatureExtractor()
features = []
img_paths = []

for feature_path in Path(".\\kaggle\\val_set_kaggle").glob("*.npy"):
    features.append(np.load(feature_path))
    img_paths.append(Path('.\\kaggle\\val_set_kaggle') / (feature_path.stem + ".png"))
    
# Insert the image query randomly
img = Image.open(".\\kaggle\\val_set_kaggle\\Covid (30).png")
# Extract its features
query = fe.extract(img)

# Calculate the similarity (distance) between images
dists = np.linalg.norm(features - query, axis = 1)
# Extract 5 images that have lowest distance
ids = np.argsort(dists)[:5]
scores = [(dists[id], img_paths[id]) for id in ids]

# Visualize the cbir result
axes=[]
fig=plt.figure(figsize=(8,8))
for index,score in enumerate(scores):
    axes.append(fig.add_subplot(1, 5, index+1))
    subplot_title=str(score[0])
    axes[-1].set_title(subplot_title)  
    plt.axis('off')
    plt.imshow(Image.open(score[1]))
fig.tight_layout()
plt.show()

images = [(img_paths[id]) for id in ids]
images = [str(image) for image in images]

imgs = manual_pre_process(images, 224)

# Visualize the five images after applying grad-cam method
plt.figure(figsize=(8, 8))

for i, img in enumerate(imgs):
    ax = plt.subplot(1, 5, i+1)
    grad_cam = make_gradcam_heatmap(np.expand_dims(img, axis=0), model, last_conv_layer)
    grad_cam_emphasized = create_gradcam(img, grad_cam, cam_path="cam.jpg", alpha=0.4)
    plt.axis('off')
    plt.imshow(grad_cam_emphasized)
plt.tight_layout()

In [ ]:
#--------------------------------------------------- Grad-CAM -----------------------------------------------------------------#

features_cam = []

# Insert the .npy file of the query image
features_cam = np.load('.\\kaggle\\grad_cam_images\\Covid (30).npy')

features_mod = features_cam * features
query_mod = query * features_cam

# Calculate the similarity (distance) between images
dists_2 = np.linalg.norm(features_mod - query_mod, axis = 1)
# Extract 5 images that have lowest distance
ids_2 = np.argsort(dists_2)[:5]
scores_2 = [(dists_2[id], img_paths[id]) for id in ids_2]

# Visualize the cbir result
axes=[]
fig=plt.figure(figsize=(8,8))
for index,score in enumerate(scores_2):
    axes.append(fig.add_subplot(1, 5, index+1))
    subplot_title=str(score[0])
    axes[-1].set_title(subplot_title)  
    plt.axis('off')
    plt.imshow(Image.open(score[1]))
fig.tight_layout()
plt.show()

images_2 = [(img_paths[id]) for id in ids_2]
images_2 = [str(image) for image in images_2]

imgs_2 = manual_pre_process(images_2, 224)

# Visualize the five images after applying grad-cam method
plt.figure(figsize=(8, 8))

for i, img in enumerate(imgs_2):
    ax = plt.subplot(1, 5, i+1)
    grad_cam = make_gradcam_heatmap(np.expand_dims(img, axis=0), model, last_conv_layer)
    grad_cam_emphasized = create_gradcam(img, grad_cam, cam_path="cam.jpg", alpha=0.4)
    plt.axis('off')
    plt.imshow(grad_cam_emphasized)
plt.tight_layout()

In [ ]:
#-------------------------------------------------- CBIR/Grad-CAM -------------------------------------------------------------#

alpha = 0.5 

dists_final = alpha * dists + dists_2
# Extract 5 images that have lowest distance
ids_final = np.argsort(dists_final)[:5]
scores_final = [(dists_final[id], img_paths[id]) for id in ids_final]

# Visualize the cbir result
axes=[]
fig=plt.figure(figsize=(8,8))
for index,score in enumerate(scores_final):
    axes.append(fig.add_subplot(1, 5, index+1))
    subplot_title=str(score[0])
    axes[-1].set_title(subplot_title)  
    plt.axis('off')
    plt.imshow(Image.open(score[1]))
fig.tight_layout()
plt.show()

images_final = [(img_paths[id]) for id in ids_final]
images_final = [str(image) for image in images_final]

imgs_final = manual_pre_process(images_final, 224)

# Visualize the five images after applying grad-cam method
plt.figure(figsize=(8, 8))

for i, img in enumerate(imgs_final):
    ax = plt.subplot(1, 5, i+1)
    grad_cam = make_gradcam_heatmap(np.expand_dims(img, axis=0), model, last_conv_layer)
    grad_cam_emphasized = create_gradcam(img, grad_cam, cam_path="cam.jpg", alpha=0.4)
    plt.axis('off')
    plt.imshow(grad_cam_emphasized)
plt.tight_layout()